In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import glob
import os

path = "/content/drive/MyDrive/MCI/KarAmouzi"
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [2]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):

    def __init__(self):
      HTMLParser.__init__(self)
      self._lines = []

    def handle_starttag(self, tag, attrs):
        self._lines.append((tag,0))


    def handle_endtag(self, tag):
        self._lines.append((tag,1))


    def handle_data(self, data):
        if(data.replace('\n','').strip() != ''):
          self._lines.append(data)


    def return_all_tags_with_data(self):
       return self._lines

In [7]:
from bs4 import BeautifulSoup
import base64 as b64
import re

descriptions = []
train_htmls = []
test_htmls = []

for i in range(3):
  df = pd.read_csv(csv_files[i], on_bad_lines='skip')
  end = 500 if len(df) > 500  else len(df)
  for j in range(end): 
    bsDecode = b64.b64decode(df['html'][j]).decode('utf-8', errors="ignore")
    soup = BeautifulSoup(bsDecode, "html.parser")
    desc = soup.findAll(attrs={"name": re.compile(r"description", re.I)}) 
    if(desc ):
      if(desc[0]['content'] not in descriptions ):
        descriptions.append(desc[0]['content'])
        if( i != 0 ):
          train_htmls.append((bsDecode,desc[0]['content']))
        else:
          test_htmls.append((bsDecode,desc[0]['content']))

In [8]:
def html_feature(z,k=0,start=-1):
  
  for i in range(len(z)):
    
    if(isinstance(z[i], tuple) and z[i][0] == 'li' and z[i][1] == 0 ):
      start = 0

    if(isinstance(z[i], tuple) and z[i][0] == 'li' and z[i][1] == 1 ):
      start = -1

    if(start == 0 ):
      continue

    if(not isinstance(z[i], tuple) and z[i-1][0] != 'script' and len(z[i].lstrip().rstrip().split(' '))>10 ):
        len_previous_text =0
        counter = 0
        for j in range(i-1,-1,-1):
          if(counter == 5):
            break
          if(not isinstance(z[j], tuple)):
            len_previous_text += len(z[j].lstrip().rstrip().split(' '))
            counter+=1

        len_next_text = 0
        counter = 0
        for j in range(i+1,len(z)):
          if(counter == 5):
            break
          if(not isinstance(z[j], tuple)):
            len_next_text += len(z[j].lstrip().rstrip().split(' '))
            counter+=1
        
        target = 0
        if(z[i] == html[1]):
          target = 1

        html_features.append([len(z[i].lstrip().rstrip().split(' ')),len_previous_text,len_next_text,k,target])
        k+=1
  return html_features

In [9]:
parser = MyHTMLParser()
train = []
test = []

for html in train_htmls:
  html_features = []
  parser.feed(html[0])
  z = parser.return_all_tags_with_data()
  k = 0
  start = -1
  html_features = html_feature(z,k=0,start=-1)
  df2 = pd.DataFrame(html_features, columns=['word_count', 'word_count_previous', 'word_count_next','position','target'])
  if(df2['target'].nunique() == 2):
      train.extend(html_features)


for html in test_htmls:
  html_features = []
  parser.feed(html[0])
  z = parser.return_all_tags_with_data()
  k = 0
  start = -1
  html_features = html_feature(z,k=0,start=-1)
  df2 = pd.DataFrame(html_features, columns=['word_count', 'word_count_previous', 'word_count_next','position','target'])
  if(df2['target'].nunique() == 2):
      test.extend(html_features)



In [10]:
import pandas as pd
df_train = pd.DataFrame(train, columns=['word_count', 'word_count_previous', 'word_count_next','position','target'])
df_train['target'].value_counts()

0    3749399
1        485
Name: target, dtype: int64

In [11]:
df_train.head()

,word_count,word_count_previous,word_count_next,position,target
0,12,1,48,0,0
1,12,11,107,1,0
2,28,21,86,2,0
3,72,50,68,3,0
4,54,84,16,4,0


In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

X = df_train[['word_count', 'word_count_previous', 'word_count_next','position']].values
y =  df_train['target'].values
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), OneClassSVM(gamma='auto', nu=0.01))
clf.fit(X, y)


In [ ]:
df_test = pd.DataFrame(test, columns=['word_count', 'word_count_previous', 'word_count_next','position','target'])
df_test['target'].value_counts()

In [102]:
X = df_test[['word_count', 'word_count_previous', 'word_count_next','position']].values
y =  df_test['target'].values

print(clf.predict(X[0]))

[0]
